DeepLarning Couse HSE 2016 fall: 
* Arseniy Ashuha, you can text me ```ars.ashuha@gmail.com```,
* ```https://vk.com/ars.ashuha``` 
* partially reusing https://github.com/ebenolson/pydata2015

<h1 align="center"> Image Captioning </h1> 

In this seminar you'll be going through the image captioning pipeline.

To begin with, let us download the dataset of image features from a pre-trained GoogleNet.



In [36]:
%env THEANO_FLAGS=device=cpu,floatX=float32

env: THEANO_FLAGS=device=cpu,floatX=float32


In [37]:
###
### Or alternatevely
### !wget https://www.dropbox.com/s/3hj16b0fj6yw7cc/data.tar.gz?dl=1 -O data.tar.gz
### !tar -xvzf data.tar.gz
###

DATA_DIR = '/wrk/trng103/'

### Data preprocessing

In [38]:
%%time
# Read Dataset
import numpy as np
import os.path as osp
import pickle

img_codes = np.load(osp.join(DATA_DIR, "data/image_codes.npy"))
captions = pickle.load(open(osp.join(DATA_DIR, 'data/caption_tokens.pcl'), 'rb'))

CPU times: user 895 ms, sys: 420 ms, total: 1.31 s
Wall time: 1.31 s


In [39]:
print ("each image code is a 1000-unit vector:", img_codes.shape)
print (img_codes[0,:10])
print ('\n\n')
print ("for each image there are 5-7 descriptions, e.g.:\n")
print ('\n'.join(captions[0]))

each image code is a 1000-unit vector: (123287, 1000)
[ 1.38901556 -3.82951474 -1.94360816 -0.5317238  -0.03120959 -2.87483215
 -2.9554503   0.6960277  -0.68551242 -0.7855981 ]



for each image there are 5-7 descriptions, e.g.:

a man with a red helmet on a small moped on a dirt road
man riding a motor bike on a dirt road on the countryside
a man riding on the back of a motorcycle
a dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud wreathed mountains
a man in a red shirt and a red hat is on a motorcycle on a hill side


In [40]:
#split descriptions into tokens
for img_i in range(len(captions)):
    for caption_i in range(len(captions[img_i])):
        sentence = captions[img_i][caption_i] 
        captions[img_i][caption_i] = ["#START#"]+sentence.split(' ')+["#END#"]


In [41]:
# Build a Vocabulary
from collections import Counter
word_counts = Counter()

#<Compute word frequencies for each word in captions. See code above for data structure>
# tip:
#word_counts.update(captions[0][0])

for img_i in range(len(captions)):
    for caption_i in range(len(captions[img_i])):
        sentence = captions[img_i][caption_i] 
        word_counts.update(sentence)

print(word_counts)



Counter({'a': 1019785, '#END#': 616767, '#START#': 616767, 'on': 224758, 'of': 212689, 'the': 206178, 'in': 191793, 'with': 161216, 'and': 146755, 'is': 102390, 'man': 75957, 'to': 71183, 'sitting': 55190, 'an': 51987, 'two': 50467, 'at': 44506, 'standing': 44297, 'people': 43707, 'are': 42776, 'next': 38867, 'white': 37898, 'woman': 35372, 'street': 31572, 'table': 31444, 'that': 29168, 'holding': 29106, 'large': 25614, 'it': 25608, 'person': 25432, 'some': 25385, 'down': 24038, 'top': 23254, 'group': 22551, 'up': 22425, 'field': 21659, 'small': 20892, 'tennis': 20556, 'near': 20432, 'front': 20342, 'black': 20288, 'his': 20242, 'train': 19464, 'plate': 19183, 'room': 19067, 'riding': 18841, 'dog': 18839, 'red': 18110, 'cat': 17908, 'young': 17804, 'by': 17762, 'water': 17096, 'baseball': 16323, 'has': 15818, 'playing': 15354, 'while': 15278, 'walking': 15182, 'bathroom': 15036, 'sign': 14666, 'blue': 14340, 'kitchen': 13917, 'food': 13851, 'grass': 13559, 'bus': 13372, 'there': 13282

In [42]:
word_counts.most_common()[::-1]

[('froth', 1),
 ('sacrificial', 1),
 ('silohuettes', 1),
 ('parachuted', 1),
 ('ncluding', 1),
 ('plutos', 1),
 ('tennnis', 1),
 ('mal', 1),
 ('cleansers', 1),
 ('diwn', 1),
 ('sorely', 1),
 ('hotdgs', 1),
 ('1:05', 1),
 ('1:14', 1),
 ('tal', 1),
 ('forter', 1),
 ('restacked', 1),
 ('mobster', 1),
 ('spiring', 1),
 ('nighstand', 1),
 ('broccolit', 1),
 ('segregated', 1),
 ('chapati', 1),
 ('passports', 1),
 ('cesars', 1),
 ('silvester', 1),
 ('througout', 1),
 ('sightings', 1),
 ('confusingly', 1),
 ('festoon', 1),
 ('ramming', 1),
 ('marty', 1),
 ('ourtside', 1),
 ('givingn', 1),
 ('grapple', 1),
 ('beear', 1),
 ('taoilet', 1),
 ('biohazard', 1),
 ('foccia', 1),
 ('combi', 1),
 ('sines', 1),
 ('aimlessly', 1),
 ('stoppes', 1),
 ('proponents', 1),
 ('fuit', 1),
 ('cowbird', 1),
 ('siitting', 1),
 ('interprets', 1),
 ('watchs', 1),
 ('fewa', 1),
 ('banned', 1),
 ('photographs:', 1),
 ('woodburning', 1),
 ('combos', 1),
 ('hlaves', 1),
 ('leaflet', 1),
 ('casenext', 1),
 ('skid', 1),
 ('

In [43]:
word_counts.most_common()

[('a', 1019785),
 ('#END#', 616767),
 ('#START#', 616767),
 ('on', 224758),
 ('of', 212689),
 ('the', 206178),
 ('in', 191793),
 ('with', 161216),
 ('and', 146755),
 ('is', 102390),
 ('man', 75957),
 ('to', 71183),
 ('sitting', 55190),
 ('an', 51987),
 ('two', 50467),
 ('at', 44506),
 ('standing', 44297),
 ('people', 43707),
 ('are', 42776),
 ('next', 38867),
 ('white', 37898),
 ('woman', 35372),
 ('street', 31572),
 ('table', 31444),
 ('that', 29168),
 ('holding', 29106),
 ('large', 25614),
 ('it', 25608),
 ('person', 25432),
 ('some', 25385),
 ('down', 24038),
 ('top', 23254),
 ('group', 22551),
 ('up', 22425),
 ('field', 21659),
 ('small', 20892),
 ('tennis', 20556),
 ('near', 20432),
 ('front', 20342),
 ('black', 20288),
 ('his', 20242),
 ('train', 19464),
 ('plate', 19183),
 ('room', 19067),
 ('riding', 18841),
 ('dog', 18839),
 ('red', 18110),
 ('cat', 17908),
 ('young', 17804),
 ('by', 17762),
 ('water', 17096),
 ('baseball', 16323),
 ('has', 15818),
 ('playing', 15354),
 ('whil

In [44]:
word_counts

Counter({'coming': 1628,
         'tartan': 2,
         'dogging': 1,
         'overhead': 571,
         'brackets': 5,
         'settin': 1,
         'selfies': 11,
         'bicuts': 1,
         'strutting': 2,
         'skateboaard': 1,
         'poeple': 5,
         'quizzical': 4,
         'gyro': 6,
         'tankers': 3,
         'braddock': 1,
         'violent': 5,
         'anemone': 1,
         'greenwich': 5,
         'fairgrounds': 2,
         'brownie': 34,
         'refidgerator': 1,
         'markham': 1,
         'weenies': 5,
         'backseats': 1,
         'chairing': 1,
         'outdoor': 1691,
         'fluids': 2,
         'corresponding': 2,
         'router': 6,
         'teddybear': 17,
         'corgi': 7,
         'petting': 483,
         'smooching': 1,
         'transfer': 4,
         'pilot': 54,
         'lofted': 1,
         'marquees': 1,
         'cascade': 4,
         'personals': 1,
         'scalpel': 3,
         'bayou': 2,
         'refridgerat

In [45]:
vocab  = ['#UNK#', '#START#', '#END#']
vocab += [k for k, v in word_counts.items() if v >= 5]
n_tokens = len(vocab)

assert 10000 <= n_tokens <= 10500

word_to_index = {w: i for i, w in enumerate(vocab)}

In [46]:
PAD_ix = -1
UNK_ix = vocab.index('#UNK#')

#good old as_matrix for the third time
def as_matrix(sequences,max_len=None):
    max_len = max_len or max(map(len,sequences))
    
    matrix = np.zeros((len(sequences),max_len),dtype='int32')+PAD_ix
    for i,seq in enumerate(sequences):
        row_ix = [word_to_index.get(word,UNK_ix) for word in seq[:max_len]]
        matrix[i,:len(row_ix)] = row_ix
    
    return matrix


In [47]:
#try it out on several descriptions of a random image
as_matrix(captions[1337])

array([[ 3731,  1251,  3439,  6882,  6717,  8526,  5506,  2738,   617,
         8009,  1148,  7911,   607,    -1,    -1],
       [ 3731,   617,  8009,  3515,  8671,  5882,  7911,  1148,  2544,
         4109,  5506,   607,    -1,    -1,    -1],
       [ 3731,  2095,  6882,  6717,  8526,  5506,  2738,   617,  3885,
         4368,  3322,  2351,  4456,  7940,   607],
       [ 3731,  2095, 10264,  2804,  6557,  9247,  2095, 10264,  2502,
          607,    -1,    -1,    -1,    -1,    -1],
       [ 3731,  1251,  3439,  6882,  6717,  8526,  5506,  2738,   617,
         8009,  3322,  2351,  7668,   607,    -1]], dtype=int32)

### Mah Neural Network

In [48]:
# network shapes. 
CNN_FEATURE_SIZE = img_codes.shape[1]
EMBED_SIZE = 128 #pls change me if u want
LSTM_UNITS = 200 #pls change me if u want

In [49]:
import theano
import theano.tensor as T

In [50]:
# Input Variable
sentences = T.imatrix()# [batch_size x time] of word ids
image_vectors = T.matrix() # [batch size x unit] of CNN image features
sentence_mask = T.neq(sentences,PAD_ix)

In [51]:
import lasagne
from lasagne.layers import *

In [52]:
#network inputs
l_words = InputLayer((None,None),sentences )
l_mask = InputLayer((None,None),sentence_mask )

#embeddings for words 
#<apply word embedding. use EMBED_SIZE>

l_word_embeddings = EmbeddingLayer(l_words,n_tokens,EMBED_SIZE)

#cudos for using some pre-trained embedding :)

In [53]:
l_word_embeddings.output_shape
CNN_FEATURE_SIZE

1000

In [54]:
# input layer for image features
l_image_features = InputLayer((None,CNN_FEATURE_SIZE),image_vectors )

#convert 1000 image features from googlenet to whatever LSTM_UNITS you have set
#it's also a good idea to add some dropout here and there

#<<convert l_image features to a shape equal to rnn hidden state. Also play with dropout/noize>

l_image_features_small = DenseLayer(l_image_features,LSTM_UNITS)

assert l_image_features_small.output_shape == (None,LSTM_UNITS)

In [55]:
# Concatinate image features and word embedings in one sequence 
decoder = LSTMLayer(l_word_embeddings,LSTM_UNITS,cell_init=l_image_features_small,mask_input=l_mask,grad_clipping=5)
#a recurrent layer (gru/lstm) with following checklist:
#    * takes word embeddings as an input
#    * has LSTM_UNITS units in the final layer
#    * has cell_init (or hid init for gru) set to converted image features
#    * mask_input = input_mask
#    * don't forget the grad clipping (~5-10)

#find out better recurrent architectures for bonus point

In [56]:
!pip3 install --upgrade https://github.com/yandexdataschool/AgentNet/archive/master.zip  tqdm --user

    100% |████████████████████████████████| 8.0MB 142kB/s eta 0:00:01
Requirement already up-to-date: tqdm in /homeappl/home/mohakala/.local/lib/python3.4/site-packages
Requirement already up-to-date: six in /homeappl/appl_taito/opt/python/3.4.5-gcc493-shared/lib/python3.4/site-packages (from agentnet==0.10.2)
Requirement already up-to-date: lasagne in /homeappl/home/mohakala/.local/lib/python3.4/site-packages (from agentnet==0.10.2)
Requirement already up-to-date: theano>=0.8.2 in /homeappl/home/mohakala/.local/lib/python3.4/site-packages (from agentnet==0.10.2)
  Using cached numpy-1.12.0-cp34-cp34m-manylinux1_x86_64.whl
Requirement already up-to-date: scipy>=0.14 in /homeappl/appl_taito/opt/python/3.4.5-gcc493-shared/lib/python3.4/site-packages/scipy-0.18.1-py3.4-linux-x86_64.egg (from theano>=0.8.2->agentnet==0.10.2)
  Found existing installation: agentnet 0.10.2
    Uninstalling agentnet-0.10.2:
      Successfully uninstalled agentnet-0.10.2
  Running setup.py install for agentnet

In [57]:
from broadcast import BroadcastLayer,UnbroadcastLayer
broadcast_decoder_ticks = BroadcastLayer(decoder,(0,1))
predicted_probabilities_flat = DenseLayer(broadcast_decoder_ticks,n_tokens,nonlinearity=lasagne.nonlinearities.softmax)
predicted_probabilities = UnbroadcastLayer(predicted_probabilities_flat,
broadcast_layer=broadcast_decoder_ticks)

In [58]:
decoder.output_shape
### Bad: predicted_probabilities=(DenseLayer(decoder,n_tokens,nonlinearity=lasagne.nonlinearities.softmax,num_leading_axes=2))

(None, None, 200)

In [59]:
predicted_probabilities.output_shape

(None, None, 10373)

In [60]:
assert predicted_probabilities.output_shape == (None, None, 10373)

In [61]:
## Decoding of rnn hiden states
#from broadcast import BroadcastLayer,UnbroadcastLayer

##apply whatever comes next to each tick of each example in a batch. Equivalent to 2 reshapes
#broadcast_decoder_ticks = BroadcastLayer(decoder,(0,1))
#print ("broadcasted decoder shape = ",broadcast_decoder_ticks.output_shape)

In [62]:
##predict probabilities for next tokens
#predicted_probabilities_each_tick = <predict probabilities for each tick, using broadcasted_decoder_shape as an input. No reshaping needed here.>
## maybe a more complicated architecture will work better?

In [63]:
##un-broadcast back into (batch,tick,probabilities)
#predicted_probabilities = UnbroadcastLayer(predicted_probabilities_each_tick,
#                                           broadcast_layer=broadcast_decoder_ticks)

#print "output shape = ",predicted_probabilities.output_shape
#
##remove if you know what you're doing (e.g. 1d convolutions or fixed shape)
#assert predicted_probabilities.output_shape == (None, None, 10373)

In [64]:
#logsoftmax

### Some tricks
* If you train large network, it is usually a good idea to make a 2-stage prediction
    1. (large recurrent state) -> (bottleneck e.g. 256)
    2. (bottleneck) -> (vocabulary size)
    * this way you won't need to store/train (large_recurrent_state x vocabulary size) matrix
* Also maybe use Hierarchical Softmax?
    * https://gist.github.com/justheuristic/581853c6d6b87eae9669297c2fb1052d




In [65]:
#next_word_probas = lasagne.layers.get_output(predicted_probabilities)
##<get network output>

#predictions_flat = next_word_probas[:,:-1].reshape((-1,n_tokens))
#reference_answers = sentences[:,1:].reshape((-1,))

##write symbolic loss function to train NN for
#loss = T.nnet.categorical_crossentropy(predictions_flat,reference_answers)  # meas-squred error would be poor 
##<compute elementwise loss function>

##mean over non-PAD tokens
#output_mask = sentence_mask[:,1:]
#loss = (loss.reshape(reference_answers.shape)*output_mask).sum() / output_mask.sum()

next_word_probas = lasagne.layers.get_output(predicted_probabilities) 
predictions_flat = next_word_probas[:,:-1].reshape((-1,n_tokens)) 
reference_answers = sentences[:,1:].reshape((-1,)) 
#write symbolic loss function to train NN for 
loss = T.nnet.categorical_crossentropy(predictions_flat,reference_answers) 
#mean over non-PAD tokens 
output_mask = sentence_mask[:,1:] 
loss = T.mean(loss*output_mask.ravel())

In [66]:
#trainable NN weights
weights = get_all_params(predicted_probabilities,trainable=True)
updates = lasagne.updates.adadelta(loss,weights)
# <parameter updates using your favorite algoritm>

In [67]:
#compile a functions for training and evaluation
#please not that your functions must accept image features as FIRST param and sentences as second one
train_step = theano.function([image_vectors,sentences],loss,updates=updates,allow_input_downcast=True)
val_step = theano.function([image_vectors,sentences],loss,allow_input_downcast=True)
#train_step = <function that takes input sentence and image mask, outputs loss and updates weights>
#val_step   = <function that takes input sentence and image mask and outputs loss>
#for val_step use deterministic=True if you have any dropout/noize

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{Cast{float32}}(TensorConstant{(1, 1) of 0})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/homeappl/home/mohakala/.local/lib/python3.4/site-packages/theano/gof/opt.py", line 1964, in process_node
    replacements = lopt.transform(node)
  File "/homeappl/home/mohakala/.local/lib/python3.4/site-packages/theano/tensor/opt.py", line 6441, in constant_folding
    no_recycling=[], impl=impl)
  File "/homeappl/home/mohakala/.local/lib/python3.4/site-packages/theano/gof/op.py", line 924, in make_thunk
    no_recycling)
  File "/homeappl/home/mohakala/.local/lib/python3.4/site-packages/theano/gof/op.py", line 828, in make_c_thunk
    output_storage=node_output_storage)
  File "/homeappl/home/mohakala/.local/lib/python3.4/site-packages/theano/gof/cc.py", line 1190, in make_thunk
    keep_lock=keep_lock)
  File "/homeappl/home/mohakal

# Training

* You first have to implement a batch generator
* Than the network will get trained the usual way

In [42]:
captions = np.array(captions)

In [43]:
from random import choice

def generate_batch(images,captions,batch_size,max_caption_len=None):
    
    #sample random numbers for image/caption indicies
    random_image_ix = np.random.randint(0,len(images),size=batch_size)
    
    #get images
    batch_images = images[random_image_ix]
    
    #5-7 captions for each image
    captions_for_batch_images = captions[random_image_ix]
    
    #pick 1 from 5-7 captions for each image
    batch_captions = map(choice,captions_for_batch_images)
    
    #convert to matrix
    batch_captions_ix = as_matrix(batch_captions,max_len=max_caption_len)
    
    return batch_images, batch_captions_ix
    

In [44]:
generate_batch(img_codes,captions,3)

TypeError: object of type 'map' has no len()

### Main loop
* We recommend you to periodically evaluate the network using the next "apply trained model" block
 *  its safe to interrupt training, run a few examples and start training again

In [68]:
batch_size=50 #adjust me
n_epochs=100 #adjust me
n_batches_per_epoch = 50 #adjust me
n_validation_batches = 5 #how many batches are used for validation after each epoch


In [69]:
!pip3 install --user tqdm

In [ ]:
from tqdm import tqdm

for epoch in range(n_epochs):
    
    train_loss=0
    for _ in tqdm(range(n_batches_per_epoch)):
        train_loss += train_step(*generate_batch(img_codes,captions,batch_size))
    train_loss /= n_batches_per_epoch
    
    val_loss=0
    for _ in range(n_validation_batches):
        val_loss += val_step(*generate_batch(img_codes,captions,batch_size))
    val_loss /= n_validation_batches
    
    print('\nEpoch: {}, train loss: {}, val loss: {}'.format(epoch, train_loss, val_loss))

print("Finish :)")

### apply trained model

In [ ]:
!pip3 install --user scikit-image

In [ ]:
#the same kind you did last week, but a bit smaller
from pretrained_lenet import build_model,preprocess,MEAN_VALUES

# build googlenet
lenet = build_model()

#load weights
#lenet_weights = pickle.load(open(osp.join(DATA_DIR, 'data/blvc_googlenet.pkl')), encoding='latin1')['param values']
lenet_weights = np.load(osp.join(DATA_DIR, 'data/blvc_googlenet.npz'), encoding='latin1')['param values']
set_all_param_values(lenet["prob"], lenet_weights)

#compile get_features
cnn_input_var = lenet['input'].input_var
cnn_feature_layer = lenet['loss3/classifier']
get_cnn_features = theano.function([cnn_input_var], lasagne.layers.get_output(cnn_feature_layer))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

#sample image
img = plt.imread(osp.join(DATA_DIR, 'data/Dog-and-Cat.jpg'))
img = preprocess(img)

In [ ]:
#deprocess and show, one line :)
from pretrained_lenet import MEAN_VALUES
plt.imshow(np.transpose((img[0] + MEAN_VALUES)[::-1],[1,2,0]).astype('uint8'))

## Generate caption

In [ ]:
last_word_probas = <get network-predicted probas at last tick
#TRY OUT deterministic=True if you want more steady results

get_probs = theano.function([image_vectors,sentences], last_word_probas)

#this is exactly the generation function from week5 classwork,
#except now we condition on image features instead of words
def generate_caption(image,caption_prefix = ("START",),t=1,sample=True,max_len=100):
    image_features = get_cnn_features(image)
    caption = list(caption_prefix)
    for _ in range(max_len):
        
        next_word_probs = <obtain probabilities for next words>
        assert len(next_word_probs.shape) ==1 #must be one-dimensional
        #apply temperature
        next_word_probs = next_word_probs**t / np.sum(next_word_probs**t)

        if sample:
            next_word = np.random.choice(vocab,p=next_word_probs) 
        else:
            next_word = vocab[np.argmax(next_word_probs)]

        caption.append(next_word)

        if next_word=="#END#":
            break
            
    return caption

In [ ]:
for i in range(10):
    print ' '.join(generate_caption(img,t=5.)[1:-1])

# Demo
### Find at least 10 images to test it on.
* Seriously, that's part of an assignment. Go get at least 10 pictures to get captioned
* Make sure it works okay on __simple__ images before going to something more comples
* Photos, not animation/3d/drawings, unless you want to train CNN network on anime
* Mind the aspect ratio (see what `preprocess` does to your image)

In [ ]:
#apply your network on image sample you found
#
#

# grading

* base 5 if it compiles and trains without exploding
* +1 for finding representative set of reference examples
* +2 for providing 10+ examples where network provides reasonable captions (at least sometimes :) )
  * you may want to predict with sample=False and deterministic=True for consistent results
  * kudos for submitting network params that reproduce it
* +2 for providing 10+ examples where network fails IF you also got previous 10 examples right


* bonus points for experiments with architecture and initialization (see above)
* bonus points for trying out other pre-trained nets for captioning
* a whole lot of bonus points if you also train via metric learning
 * image -> vec
 * caption -> vec (encoder, not decoder)
 * loss = correct captions must be closer, wrong ones must be farther
 * prediction = choose caption that is closest to image
* a freaking whole lot of points if you also obtain statistically signifficant results the other way round
 * take caption, get closest image